In [1]:
import networkx as nx
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import NEMtropy as nem
import numpy as np

In [2]:
# Load the datasets from the assignment_1_data folder
path = "NYSE/"
npy_files = [f for f in listdir(path) if isfile(join(path, f)) and f.endswith(".npy")]
numpy_files = [np.load(path + f) for f in npy_files]

correlation_matrices = {k: v for k, v in zip(npy_files, numpy_files)}
stocknames = [np.loadtxt(path + "stocknames.txt", dtype=str)]